## Imports

In [17]:
import pandas as pd
import numpy as np

import nltk
import os
import re
import tensorflow as tf

import spacy
from tensorboard.plugins import projector
import matplotlib.pyplot as plt

2020-11-18 19:17:29,443 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


#### Reading the File

In [2]:
df = pd.read_csv('tweets_clean.csv')
df.head()

,id,author_id,body,country,publishing_date,coordinates,place,user_country,retweet_count,url,...,name,favourites_count,friends_count,location,extracted_country,created_at,listed_count,project_tweet_count,rtt_author_ids,retweet_dates
0,4cf9943a-0c79-11eb-8577-02420a00006f,18657350,Ça se passe possiblement en Chine (ou un quart...,None,2020-10-04 16:28:42,None,None,CA,0,https://twitter.com/Vero_Meunier/status/131279...,...,Véronique Meunier,14942,2916,"Montréal, Canada",CA,2009-01-06 02:13:04,33,1,NaN,NaN
1,54b589b8-0c79-11eb-8577-02420a00006f,814383841855045633,@Merovingien481 @XuluQen @Mediavenir Pour se f...,None,2020-10-04 15:27:36,None,None,FR,0,https://twitter.com/Nzonzimi/status/1312776392...,...,Nzonzimi 🌍,134222,413,"Belfort, France",FR,2016-12-29 08:13:33,0,14,NaN,NaN
2,e49281dc-0c77-11eb-8577-02420a00006f,891464784,Ptdrrrrrr la frappe de Chine je suis morte htt...,None,2020-10-07 08:35:00,None,None,None,0,https://twitter.com/SassouwQb/status/131375971...,...,Dounias,879,118,NaN,None,2012-10-19 17:46:58,0,1,NaN,NaN
3,b58b95b6-0c79-11eb-8577-02420a00006f,1057778746839760896,@aliyya213s Que Dieu te pardonne 🙏. Genre il y...,None,2020-10-03 21:23:08,None,None,None,0,https://twitter.com/hay_dera/status/1312503476...,...,🙏🙏🙏,721,1429,NaN,None,2018-10-31 23:38:00,2,1,NaN,NaN
4,9aeb11de-0c77-11eb-8577-02420a00006f,724288676,« La frappe de Chine » ptdrrrr ils sont ravagé...,France,2020-10-07 11:49:10,None,Les Mureaux,FR,4,https://twitter.com/Sergueyui/status/131380858...,...,ّ,37849,99,NaN,FR,2012-07-29 16:47:43,5,1,"[4351389615, 919529679484342272, 1863693260, 1...",['12875 2020-10-07 11:49:47\nName: retweete...


## Preprocessing

In [3]:
def get_hyperlinks(text):
    ## finds all hyperlinks in the text
    return re.findall(r"http\S+", text)

    def rmv_hyperlink(text):
        ## removes all hyperlinks of the text
        return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)

def get_hashtags(text):
    ## finds all hashtags of the text
    return re.findall(r"#\S+", text)

def rmv_hashtags(text):
    ## removes all hashtags of the text
    return re.sub(r"#\S+", "", text)

def get_users(text):
    return re.findall(r"@\S+", text)

def scrap_tweets(df):
    df['hyperlinks'] = df['body'].map(
        get_hyperlinks
    )
    df['Title'] = df['body'].map(
        rmv_hyperlink
    )
    df['hashtags'] = df['body'].map(
        get_hashtags
    )
    df['Title'] = df['body'].map(
        rmv_hashtags
    )
    df['referenced_users'] = df['body'].map(
        get_users
    )

In [4]:
def clean_tweets(df):
    res = df# df[df['user_country'] == 'FR']
    P = (len(df) - len(res)) / len(df)
    print('{}% of tweets were removed'.format(P))
    return res
    
only_FR_df = clean_tweets(df)

0.0% of tweets were removed


#### tokenizing in sentences and PoS

In [5]:
## french PoS:
## https://github.com/cmchurch/nltk_french/blob/master/french-nltk.py

## https://nlp.stanford.edu/software/CRF-NER.html

nlp = spacy.load('fr_core_news_sm')

serie = only_FR_df['body'].map(
    rmv_hyperlink
)

### Creating vocabulary

#### Lemmatize

In [6]:
from spacy_lefff import LefffLemmatizer, POSTagger

pos = POSTagger()

french_lemmatizer = LefffLemmatizer(after_melt = True)

nlp.add_pipe(pos, name='pos', after='parser')
nlp.add_pipe(french_lemmatizer, name='lefff', after='pos')

2020-11-18 19:04:52,536 - spacy_lefff.downloader - INFO - data already set up
2020-11-18 19:04:52,537 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading lexicon...
2020-11-18 19:04:52,965 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading tags...
2020-11-18 19:04:52,987 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr...
2020-11-18 19:04:53,763 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr: done
2020-11-18 19:04:53,763 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-11-18 19:04:53,764 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-11-18 19:04:54,183 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [7]:
def lemmatize(text):
    return [doc.lemma_ for doc in nlp(text)]
    

lemmatized = serie.map(
    lemmatize
)

2020-11-18 19:04:54,218 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,235 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,249 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,267 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,286 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,302 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,316 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,329 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,338 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,351 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,368 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:54,395 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:04:56,573 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,603 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,632 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,663 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,690 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,708 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,739 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,763 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,782 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,808 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,837 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:56,864 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:04:58,983 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,004 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,019 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,032 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,055 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,090 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,125 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,153 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,170 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,188 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,205 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:04:59,234 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:05:01,304 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,317 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,341 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,374 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,391 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,421 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,462 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,492 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,515 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,536 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,561 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:01,584 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:05:03,657 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,679 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,698 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,714 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,735 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,748 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,773 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,803 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,839 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,874 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,908 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:05:03,942 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

...

2020-11-18 19:14:24,878 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,892 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,903 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,924 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,953 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,977 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:24,995 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:25,018 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:25,027 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:25,043 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:25,063 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:25,077 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:14:26,792 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,812 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,824 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,850 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,872 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,904 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,930 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,954 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,968 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,979 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:26,990 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:27,005 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:14:28,887 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,902 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,920 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,948 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,964 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,977 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:28,996 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:29,027 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:29,048 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:29,067 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:29,079 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:29,102 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:14:30,800 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,814 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,833 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,857 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,872 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,897 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,908 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,928 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,957 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,974 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:30,992 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:31,020 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-11-18 19:14:32,898 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:32,925 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:32,946 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:32,970 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:32,986 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,004 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,020 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,042 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,059 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,089 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,118 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-11-18 19:14:33,139 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

In [8]:
for l in lemmatized[:20]:
    print(l)

['ça', 'se', 'passer', 'possiblement', 'en', 'Chine', '(', 'ou', 'un', 'quartier', 'chinois', ')', 'avec', 'un', 'thème', 'musical', 'de', 'yodle', '!', 'Hahahahaha', ' ']
['@merovingien481', '@xuluqen', '@mediavenir', 'pour', 'se', 'faire', 'recoloniser', 'mais', 'par', 'le', 'Chine', 'ce', 'fois', '-', 'ci']
['Ptdrrrrrr', 'le', 'frappe', 'de', 'Chine', 'je', 'être', 'mort', ' ']
['@aliyya213s', 'que', 'Dieu', 'te', 'pardonne', '🙏', '.', 'Genre', 'il', 'y', 'avoir', 'un', 'moment', '!', '!', 'en', 'chine', 'le', 'chrétien', 'être', 'très', 'persécuter', 'aussi', '!', '!', 'falloir', 'pas', 'faire', 'l’', 'aveugle']
['«', '  ', 'le', 'frappe', 'de', 'Chine', '  ', '»', 'ptdrrrr', 'il', 'être', 'ravager', ' ']
['@charly_chin', '@Kipithrill', '@BriscoPaPa', 'dommage', 'car', 'en', 'Raid', '0', 'tu', 'avoir', 'devoir', 'avoir', 'un', 'temps', 'ultra', 'rapide']
['Chine', '–', 'Ouïgours', ':', 'l’', 'asservissement', 'd’', 'un', '  ', 'peuple', ' ']
['@mackencee', '@vincentglad', 'sans', '

In [9]:
lemmatized_joined = lemmatized.map(
    lambda x : " ".join(x)
)
lemmatized_joined

0        ça se passer possiblement en Chine ( ou un qua...
1        @merovingien481 @xuluqen @mediavenir pour se f...
2              Ptdrrrrrr le frappe de Chine je être mort  
3        @aliyya213s que Dieu te pardonne 🙏 . Genre il ...
4        «    le frappe de Chine    » ptdrrrr il être r...
                               ...                        
27309    @Hamza01180412 @nastym00n @devdetp aller ramen...
27310    ce être svt le homme à son tête qui faire un n...
27311    @valerieboyer13 pas de quoi en être fier !   l...
27312    ah non c’ être vrai , dsl , il falloir faire e...
27313    le Chine dépasser le 100 million de abonné 5 g...
Name: body, Length: 27314, dtype: object

#### Removing unwanted characters

In [10]:
def rmv_char(text):
    ## removes all unwanted characters of the text
    text = re.sub("""[\n´]""", " ", text)
    text = re.sub("""["'+"*%&/()"=¦@#°§¬|¢\[\]\-\_\—‘’“”•`\^{}~¥©?.,:!$;«»<>]""", "", text)
    return re.sub("""\d""", " ", text)

In [11]:
filtered = lemmatized_joined.map(
    lambda r: r.lower()
).map(
    rmv_char
)
filtered

0        ça se passer possiblement en chine  ou un quar...
1        merovingien    xuluqen mediavenir pour se fair...
2              ptdrrrrrr le frappe de chine je être mort  
3        aliyya   s que dieu te pardonne 🙏  genre il y ...
4            le frappe de chine     ptdrrrr il être rav...
                               ...                        
27309    hamza         nastym  n devdetp aller ramener ...
27310    ce être svt le homme à son tête qui faire un n...
27311    valerieboyer   pas de quoi en être fier    lui...
27312    ah non c être vrai  dsl  il falloir faire en b...
27313    le chine dépasser le     million de abonné   g...
Name: body, Length: 27314, dtype: object

#### Tokenizing and Removing stopwords

In [12]:
from nltk.corpus import stopwords

stopwords = stopwords.words('french')

def rmv_stop_word(tokens):
    return [x for x in tokens if x not in stopwords]

def filter_characters(tokens):
    return [x for x in tokens if x.isalpha()]

tokens = filtered.map(
    nltk.word_tokenize
).map(
    rmv_stop_word
).map(
    filter_characters
)

tokens

0        [ça, passer, possiblement, chine, quartier, ch...
1        [merovingien, xuluqen, mediavenir, faire, reco...
2                   [ptdrrrrrr, frappe, chine, être, mort]
3        [aliyya, dieu, pardonne, genre, avoir, moment,...
4                  [frappe, chine, ptdrrrr, être, ravager]
                               ...                        
27309    [hamza, nastym, devdetp, aller, ramener, cul, ...
27310    [être, svt, homme, tête, faire, nation, russie...
27311    [valerieboyer, quoi, être, fier, avoir, devoir...
27312    [ah, non, être, vrai, dsl, falloir, faire, bar...
27313    [chine, dépasser, million, abonné, gramme, tec...
Name: body, Length: 27314, dtype: object

In [13]:
def do(tokens, to_rmv):
    for x in to_rmv:
        if x in tokens:
            tokens.remove(x)
            
    return tokens

In [14]:
def build_vocab(posts):
    
    T = {}
    for tokens in posts:
        for token in tokens:
            if token in T.keys():
                T[token] = T[token] + 1
            else:
                T[token] = 1
                
    to_rmv = []
    for (w, n) in T.items():
        if n == 1:
            to_rmv.append(w)
        
    return posts.map(
        lambda l : do(l, to_rmv)
    )
    

In [15]:
tokens = build_vocab(tokens)

In [16]:
for i in tokens.index[:20]:
    print('--------')
    print(df['body'][i])
    print(tokens[i])

--------
Ça se passe possiblement en Chine (ou un quartier chinois) avec un thème musical de yodle! Hahahahaha https://t.co/6u9t268sTv
['ça', 'passer', 'possiblement', 'chine', 'quartier', 'chinois', 'thème', 'musical', 'hahahahaha']
--------
@Merovingien481 @XuluQen @Mediavenir Pour se faire recoloniser mais par la Chine cette fois-ci
['xuluqen', 'mediavenir', 'faire', 'chine', 'fois', 'ci']
--------
Ptdrrrrrr la frappe de Chine je suis morte https://t.co/5TyiXslVCI
['ptdrrrrrr', 'frappe', 'chine', 'être', 'mort']
--------
@aliyya213s Que Dieu te pardonne 🙏. Genre il y a des moment !! En chine les chrétiens sont très persécuter aussi !! Faut pas faire l’aveugle
['dieu', 'pardonne', 'genre', 'avoir', 'moment', 'chine', 'chrétien', 'être', 'très', 'persécuter', 'aussi', 'falloir', 'faire', 'aveugle']
--------
« La frappe de Chine » ptdrrrr ils sont ravagés https://t.co/Js6xNxBIPx
['frappe', 'chine', 'ptdrrrr', 'être', 'ravager']
--------
@charly_chine @Kipithrill @BriscoPaPa Dommage car

## Trying out gensim

In [20]:
vector_sizes = [36, 50, 100, 200, 300]
from gensim.models import Word2Vec

i = 0

for size in vector_sizes:
    model = Word2Vec(
        sentences = tokens,
        size = size,
        window = 5,
        workers = 4,
        min_count = 1
    )
    
    vocab = model.wv.vocab.keys()
    embeddings = model[vocab]
    
    
    log_dir = os.path.join('logs', 'gensim_{}'.format(i))
    i = i + 1
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)


    with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
        for word in vocab:
            try:
                f.write("{}\n".format(word))
            except UnicodeEncodeError:
                f.write("{}\n".format('unknown'))

    weights = tf.Variable(embeddings)
    checkpoint = tf.train.Checkpoint(embeddings = weights)
    checkpoint.save(os.path.join(log_dir, 'embedding.ckpt'))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()

    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'metadata.tsv'

2020-11-18 19:18:09,517 - gensim.models.word2vec - INFO - collecting all words and their counts
2020-11-18 19:18:09,518 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-18 19:18:09,546 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 164853 words, keeping 14770 word types
2020-11-18 19:18:09,576 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 318357 words, keeping 18984 word types
2020-11-18 19:18:09,598 - gensim.models.word2vec - INFO - collected 19970 word types from a corpus of 431855 raw words and 27314 sentences
2020-11-18 19:18:09,600 - gensim.models.word2vec - INFO - Loading a fresh vocabulary
2020-11-18 19:18:09,624 - gensim.models.word2vec - INFO - effective_min_count=1 retains 19970 unique words (100% of original 19970, drops 0)
2020-11-18 19:18:09,624 - gensim.models.word2vec - INFO - effective_min_count=1 leaves 431855 word corpus (100% of original 431855, drops 0)


c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


2020-11-18 19:18:15,369 - gensim.models.base_any2vec - WARNING - consider setting layer size to a multiple of 4 for greater performance
2020-11-18 19:18:15,370 - gensim.models.word2vec - INFO - collecting all words and their counts
2020-11-18 19:18:15,371 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-18 19:18:15,404 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 164853 words, keeping 14770 word types
2020-11-18 19:18:15,442 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 318357 words, keeping 18984 word types
2020-11-18 19:18:15,471 - gensim.models.word2vec - INFO - collected 19970 word types from a corpus of 431855 raw words and 27314 sentences
2020-11-18 19:18:15,472 - gensim.models.word2vec - INFO - Loading a fresh vocabulary
2020-11-18 19:18:15,496 - gensim.models.word2vec - INFO - effective_min_count=1 retains 19970 unique words (100% of original 19970, drops 0)
2020-

2020-11-18 19:18:20,359 - gensim.models.word2vec - INFO - Loading a fresh vocabulary
2020-11-18 19:18:20,381 - gensim.models.word2vec - INFO - effective_min_count=1 retains 19970 unique words (100% of original 19970, drops 0)
2020-11-18 19:18:20,382 - gensim.models.word2vec - INFO - effective_min_count=1 leaves 431855 word corpus (100% of original 431855, drops 0)
2020-11-18 19:18:20,431 - gensim.models.word2vec - INFO - deleting the raw counts dictionary of 19970 items
2020-11-18 19:18:20,432 - gensim.models.word2vec - INFO - sample=0.001 downsamples 24 most-common words
2020-11-18 19:18:20,433 - gensim.models.word2vec - INFO - downsampling leaves estimated 366046 word corpus (84.8% of prior 431855)
2020-11-18 19:18:20,475 - gensim.models.base_any2vec - INFO - estimated required memory for 19970 words and 100 dimensions: 25961000 bytes
2020-11-18 19:18:20,475 - gensim.models.word2vec - INFO - resetting layer weights
2020-11-18 19:18:23,823 - gensim.models.base_any2vec - INFO - trainin

2020-11-18 19:18:25,534 - gensim.models.base_any2vec - INFO - estimated required memory for 19970 words and 200 dimensions: 41937000 bytes
2020-11-18 19:18:25,536 - gensim.models.word2vec - INFO - resetting layer weights
2020-11-18 19:18:28,828 - gensim.models.base_any2vec - INFO - training model with 4 workers on 19970 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-11-18 19:18:29,074 - gensim.models.base_any2vec - DEBUG - job loop exiting, total 44 jobs
2020-11-18 19:18:29,103 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 11 jobs
2020-11-18 19:18:29,105 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 11 jobs
2020-11-18 19:18:29,109 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-11-18 19:18:29,111 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-11-18 19:18:29,111 - gensim.models.base_any2vec - DEBUG - worker e

2020-11-18 19:18:34,880 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 11 jobs
2020-11-18 19:18:34,881 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 11 jobs
2020-11-18 19:18:34,881 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-11-18 19:18:34,888 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2020-11-18 19:18:34,893 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 11 jobs
2020-11-18 19:18:34,894 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2020-11-18 19:18:34,895 - gensim.models.base_any2vec - INFO - EPOCH - 1 : training on 431855 raw words (366098 effective words) took 0.4s, 839400 effective words/s
2020-11-18 19:18:35,225 - gensim.models.base_any2vec - DEBUG - job loop exiting, total 44 jobs
2020-11-18 19:18:35,271 - gensim.models.base_any2vec - DEBUG - worker exiting, processed 1

In [173]:
model.wv.most_similar(positive = ['innovation'], negative = [], topn = 20)

[('fds', 0.9998842477798462),
 ('numérique', 0.9998542666435242),
 ('développement', 0.9998494386672974),
 ('visite', 0.9998372793197632),
 ('public', 0.9998356103897095),
 ('université', 0.9998341202735901),
 ('octobre', 0.9998316764831543),
 ('o', 0.99982088804245),
 ('g', 0.9998190999031067),
 ('travail', 0.9998161792755127),
 ('x', 0.999809980392456),
 ('chercheur', 0.9998082518577576),
 ('conférence', 0.999807596206665),
 ('a', 0.9998053312301636),
 ('laboratoir', 0.9998041987419128),
 ('cnr', 0.9998031854629517),
 ('h', 0.999801516532898),
 ('mise', 0.9997997283935547),
 ('inserm', 0.9997937679290771),
 ('social', 0.9997921586036682)]

In [164]:
from nltk.cluster import KMeansClusterer

NUM_CLUSTERS = 21

kclusterer = KMeansClusterer(
    NUM_CLUSTERS,
    distance = nltk.cluster.util.cosine_distance,
    repeats = 3
)
assigned_clusters = kclusterer.cluster(
    model[model.wv.vocab],
    assign_clusters = True
)

c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: 

In [42]:
V = np.array(list(model.wv.vocab.keys()))
K = np.array(assigned_clusters)

In [43]:
for i in range(NUM_CLUSTERS):
    print(V[K == i])

['mediavenir' 'recoloniser' 'ci' ... 'regulation' 'joestj' 'droiteil']
['ça' 'passer' 'chinois' ... 'foudroyant' 'httpstcodwygqoaexh'
 'virtuellement']
['raid' 'ultra' 'surpasser' ... 'maso' 'incompéter' 'httpstcofrxappfmnq']
['possiblement' 'dommage' 'rapide' ... 'merveilleusement' 'carriere' 'kvy']
['chine' 'merovingien' 'chrétien' ... 'httpstcowtcwupvru' 'originellement'
 'musulmanie']
['ouïgours' 'region' 'asie' ... 'uunxkg' 'cureathee' 'poltron']
['pardonne' 'charlychin' 'iqhzg' ... 'httpstcouzp' 'xuaneszv' 'terrerares']
['quartier' 'thème' 'stv' ... 'rbi' '𝗛' 'devdetp']
['u' 'xnxbipx' 'kipithrill' ... 'shawn' 'fareshas' 'alyx']
['mort' 'wuhan' 'covid' ... 'boummm' 'bécasse' 'httpstcolrvioruikc']
['génétique' 'chercheur' 'voeu' ... 'jeangalano' 'spécifiquement'
 'expliquerait']
['httpstco' 'frappe' 'ptdrrrr' ... 'danielb' 'zdedededex' 'bruceduvic']
['gilford' 'information' 'disposer' ... 'jimcameron' 'clandestino' 'aisha']
['h' 'extraodinair' 'nuit' ... 'cuve' 'maure' 'departement

## Keras Pipeline

In [18]:
import tensorflow
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tokens = tokens.map(
    lambda l : ' '.join(l)
)

2020-11-17 16:46:46,761 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [19]:
training_data = tokens.to_numpy(dtype = str)

In [20]:
## Note: can add ngrams if needed
vectorizer = TextVectorization(output_mode = 'int')
## will learn vocab
vectorizer.adapt(training_data)
integer_data = vectorizer(training_data)

print(integer_data[0])

tf.Tensor(
[5295  285    5    3  135 1835    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0], shape=(42,), dtype=int64)


In [21]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'être',
 'chine',
 'avoir',
 'faire',
 'laboratoire',
 'plus',
 'génétique',
 'tout',
 'httpstco',
 'cela',
 'france',
 'covid',
 'aller',
 'chinois',
 'comme',
 'pays',
 'pouvoir',
 'via',
 'si',
 'dire',
 'bien',
 'virus',
 'vouloir',
 'français',
 'voir',
 'monde',
 'aussi',
 'falloir',
 'après',
 'pourcent',
 'cas',
 'vie',
 'non',
 'autre',
 'contre',
 'wuhan',
 'quand',
 'nouveau',
 'savoir',
 'sans',
 'depuis',
 'très',
 'devoir',
 'état',
 'an',
 'donc',
 'rien',
 'grand',
 'entre',
 'alors',
 'bon',
 'premier',
 'h',
 'russie',
 'passer',
 'oui',
 'frappe',
 'encore',
 'là',
 'jour',
 'gouvernement',
 'peu',
 'europe',
 'comment',
 'nobel',
 'où',
 'lien',
 'personne',
 'celui',
 'usa',
 'venir',
 'mondial',
 'institut',
 'pasteur',
 'explication',
 'vidéo',
 'mettre',
 'recherche',
 'moins',
 'année',
 'politique',
 'camp',
 'fait',
 'a',
 'deux',
 'trump',
 'limiter',
 'ouïghour',
 'prix',
 'prendre',
 'déjà',
 'toujours',
 'petit',
 'concevoir',
 'chimie',
 

In [22]:
vectorizer(['fois', 'port']).numpy().ravel()

array([135, 978], dtype=int64)

In [23]:
vocab_size = len(vectorizer.get_vocabulary())
vocab_size

8125

In [24]:
vocab = vectorizer.get_vocabulary()
reverse_mapping = {}
for i in range(vocab_size):
    reverse_mapping[i] = vocab[i]

In [25]:
sampling_table = tensorflow.keras.preprocessing.sequence.make_sampling_table(vocab_size)
# Should add some 0 padding such that the largest tweet doesn't get skipgramed with the next
(data_tuples, labels) = tensorflow.keras.preprocessing.sequence.skipgrams(
    integer_data.numpy().ravel(),
    vocab_size,
    window_size = 5,
    shuffle = True,
    sampling_table = sampling_table
)

# data_tuples = np.array(data_tuples)
# labels = np.array(labels)

In [26]:
d = [[int(x[0]), int(x[1])] for x in data_tuples]
labels = np.array([int(x) for x in labels])

target_words = np.array(d)[:,0]
context_words = np.array(d)[:,1]

In [27]:
assert(len(target_words) == len(labels))

In [28]:
M = np.sum(sampling_table)
np.random.choice(vocab, 5, replace = False, p = sampling_table / M)

array(['japan', 'claudedanthony', 'yoanncolin', 'commission',
       'lordmahammer'], dtype='<U66')

## Model
### Implementation

In [48]:
class Embedding_model():
    
    def __init__(self):
        pass
    
    def build(self, vocab_size, embeddings_dim = 100):
        # input layer
        self.in_1 = tensorflow.keras.Input(1)
        self.in_2 = tensorflow.keras.Input(1)

        # embedding layer
        E = layers.Embedding(
            vocab_size,
            embeddings_dim,
            embeddings_regularizer = tensorflow.keras.regularizers.l1(l1=0.01),
            embeddings_constraint = tensorflow.keras.constraints.UnitNorm(axis=1),
            input_length = 1,
            name = 'embeddings'
        )

        # applying embedding on input
        E_1 = E(self.in_1)
        E_2 = E(self.in_2)

        D = tensorflow.keras.backend.sum(E_1 * E_2, axis = -1)

        # apply sigmoid activation function
        self.S = layers.Activation(activation = 'sigmoid')(D)
        
    def compile_(self, learning_rate = 0.1):
        self.model = tensorflow.keras.Model(
            inputs = [self.in_1, self.in_2],
            outputs = [self.S]
        )
        self.model.compile(
            loss = 'binary_crossentropy',
            optimizer = tensorflow.keras.optimizers.RMSprop(learning_rate = learning_rate)
        )
        
    def train(self, target_words, context_words, labels, batch_size = 32, epochs = 30, callbacks = [], validation_set = None):
        history = self.model.fit(
            [target_words, context_words],
            labels,
            batch_size = batch_size,
            epochs = epochs,
            callbacks = callbacks,
            shuffle = True
        )
        train_loss = history.history['loss']
        plt.figure()
        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.plot(train_loss)
        plt.show()

In [49]:
embeddings_dim = 100
learning_rate = 0.1

EM = Embedding_model()
EM.build(vocab_size, embeddings_dim = embeddings_dim)

EM.compile_(learning_rate = learning_rate)

EM.model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1, 100)       812500      input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_Mul_4 (TensorFlowOp [(None, 1, 100)]     0           embeddings[0][0]      

### Validation callback

In [50]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'être',
 'chine',
 'avoir',
 'faire',
 'laboratoire',
 'plus',
 'génétique',
 'tout',
 'httpstco',
 'cela',
 'france',
 'covid',
 'aller',
 'chinois',
 'comme',
 'pays',
 'pouvoir',
 'via',
 'si',
 'dire',
 'bien',
 'virus',
 'vouloir',
 'français',
 'voir',
 'monde',
 'aussi',
 'falloir',
 'après',
 'pourcent',
 'cas',
 'vie',
 'non',
 'autre',
 'contre',
 'wuhan',
 'quand',
 'nouveau',
 'savoir',
 'sans',
 'depuis',
 'très',
 'devoir',
 'état',
 'an',
 'donc',
 'rien',
 'grand',
 'entre',
 'alors',
 'bon',
 'premier',
 'h',
 'russie',
 'passer',
 'oui',
 'frappe',
 'encore',
 'là',
 'jour',
 'gouvernement',
 'peu',
 'europe',
 'comment',
 'nobel',
 'où',
 'lien',
 'personne',
 'celui',
 'usa',
 'venir',
 'mondial',
 'institut',
 'pasteur',
 'explication',
 'vidéo',
 'mettre',
 'recherche',
 'moins',
 'année',
 'politique',
 'camp',
 'fait',
 'a',
 'deux',
 'trump',
 'limiter',
 'ouïghour',
 'prix',
 'prendre',
 'déjà',
 'toujours',
 'petit',
 'concevoir',
 'chimie',
 

In [62]:
validation_set = ['président', 'positif', 'peuple', 'innovation']
validation_set = [vectorizer.get_vocabulary().index(w) for w in validation_set]
validation_set

[221, 424, 379, 383]

In [68]:
NUM_TOPN = 7

class ValidationCallback(tensorflow.keras.callbacks.Callback):
    
    def __init__(self, validation_set, model, reverse_mapping, NUM_TOPN = 10):
        self.validation_set = validation_set
        self.NUM_TOPN = NUM_TOPN
        self.model = model
        self.reverse_mapping = reverse_mapping
        super().__init__()
    
    def on_epoch_end(self, epoch, logs=None):
        M = self.model.get_layer('embeddings').weights[0].numpy()
        M = M / np.linalg.norm(M, axis = 1)[:, np.newaxis]
        val_vects = M[validation_set]
        
        closest = np.argsort(np.cos(M @ val_vects.T), axis = 0)
        
        top = closest[:self.NUM_TOPN, :]
        for (i,e) in enumerate(self.validation_set):
            print('\n word: {}'.format(self.reverse_mapping[e]))
            print('closest words: ')
            string = ''
            for c in top[:,i]:
                string = string + ' ' + self.reverse_mapping[c]
            print(string)
            print('----------------')
            
reduce_LR = tensorflow.keras.callbacks.ReduceLROnPlateau(
    monitor = 'loss',
    factor = 0.1,
    min_lr = 0
)

In [71]:
dims = [300]
lrs = [0.1]

for dim in dims:
    for lr in lrs:
        print('dim : {} - lr : {}'.format(dim, lr))
        EM = Embedding_model()
        EM.build(vocab_size, embeddings_dim = dim)
        EM.compile_(learning_rate = lr)
        EM.train(
            target_words,
            context_words,
            labels,
            epochs = 10,
            callbacks = [
                ValidationCallback(validation_set, EM.model, reverse_mapping, NUM_TOPN),
                reduce_LR
            ]
        )

dim : 300 - lr : 0.1
Epoch 1/10
 2256/10956 [=====>........................] - ETA: 4:14 - loss: 1220.7515

KeyboardInterrupt: 